In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Cooktop Compatibility'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Hair Styling Tools"""

regex_pattern=r'(?i)(cord length measures\s?\d+.?\d*.?\")|((cord length measures\s?\d+.?\d*.?ft))()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
df.head(1)

product_id external_id  \
0  1211525005    69581629   

                                                 product_name  \
0  Le Creuset® Demi 1.25 qt. Whistling Tea Kettle in Licorice   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0  The Demi Tea Kettle from Le Creuset features a locking handle for added control and a heat-resistant phenolic knob handle for safe use. This enamel-on-steel tea kettle also features a harmonic whistle and can be used on all heat sources. Beautiful and eye-catching design would look great on any cooktop, kitchen counter, or coffee table Durable enamel-on-steel design with flat base for rapid heating Compatible with all heat sources, including induction Fixed harmonic whistle audibly alerts you when water has reached a roiling boil Spout easily flips open for easy pouring Locking handle with heat-resistant grip for added control and safe handling Wide, snug-fitting lid with phenolic knob handle for easy interior cleaning Enameled steel construction Hand wash 1.25 qt. capacity Measures 9.5\" L x 6\" W x 4.38\" H Weighs 2.4 lb. 5-year limited manufacturer's warranty Imported   

   customer_id attributes               buckets values  
0            5       None  Teapots & Teakettles   None

In [5]:
# error

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Electric', 'Gas', 'Glass Top', 'Induction']

In [7]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0                                       [Induction]
1                                                []
2                            [Induction, Induction]
3                            [Induction, Induction]
4                                       [Induction]
                           ...                     
121                                     [Induction]
122                                              []
123                                              []
124                                     [Induction]
125    [Induction, Induction, Induction, Induction]
Name: long_desc, Length: 126, dtype: object

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df=df[df['matches'].astype(str)!='[]']
df['matches'].explode().value_counts()

Induction    120
Electric       8
Gas            6
Name: matches, dtype: int64

In [9]:
# error

In [10]:
matches=df
# matches['matches']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches.head(1)

product_id external_id  \
0  1211525005    69581629   

                                                 product_name  \
0  Le Creuset® Demi 1.25 qt. Whistling Tea Kettle in Licorice   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0  The Demi Tea Kettle from Le Creuset features a locking handle for added control and a heat-resistant phenolic knob handle for safe use. This enamel-on-steel tea kettle also features a harmonic whistle and can be used on all heat sources. Beautiful and eye-catching design would look great on any cooktop, kitchen counter, or coffee table Durable enamel-on-steel design with flat base for rapid heating Compatible with all heat sources, including induction Fixed harmonic whistle audibly alerts you when water has reached a roiling boil Spout easily flips open for easy pouring Locking handle with heat-resistant grip for added control and safe handling Wide, snug-fitting lid with phenolic knob handle for easy interior cleaning Enameled steel construction Hand wash 1.25 qt. capacity Measures 9.5\" L x 6\" W x 4.38\" H Weighs 2.4 lb. 5-year limited manufacturer's warranty Imported   

   customer_id attributes               buckets values      matches  
0            5       None  Teapots & Teakettles   None  [Induction]

In [11]:
matches['matches'].explode().value_counts()

Induction    120
Electric       8
Gas            6
Name: matches, dtype: int64

In [17]:
# test_list=pd.DataFrame()
test_list=matches['matches']
test_list[0:50]

0                                      [Induction]
2                           [Induction, Induction]
3                           [Induction, Induction]
4                                      [Induction]
5                       [Electric, Gas, Induction]
6                                      [Induction]
7                           [Induction, Induction]
8                                      [Induction]
9                           [Induction, Induction]
10                                     [Induction]
11                          [Induction, Induction]
13                          [Induction, Induction]
18                          [Induction, Induction]
19                          [Induction, Induction]
20                          [Induction, Induction]
21                          [Induction, Induction]
22                          [Induction, Induction]
24                                     [Induction]
25                                     [Induction]
26                             

In [26]:
matches['matches']=[list(set(sublist)) for sublist in matches['matches']]
matches

product_id external_id  \
0    1211525005    69581629   
2    1174632103    69700513   
3     817553150    64521749   
4     817402026    68048099   
5     817187078    62290678   
..          ...         ...   
118  1184066770    18752697   
120  1192334030    69597868   
121  1184011997    66717515   
124   817164556    43267903   
125   988369545    64329406   

                                                                      product_name  \
0                       Le Creuset® Demi 1.25 qt. Whistling Tea Kettle in Licorice   
2                Le Creuset® Signature 10.25-Inch Iron Handle Skillet in Artichaut   
3        Ayesha Curry™ Cast Iron Enamel 12-Inch Skillet with Helper Handle in Teal   
4    Rachael Ray™ Create Delicious Nonstick 12.5-Inch Aluminum Deep Skillet in Red   
5                                 Rosle Stainless Steel Nonstick 11-Inch Crepe Pan   
..                                                                             ...   
118  Circulon® Symmetry™ Nonstick Hard Anodized 14-Inch Open Stir Fry Pan in Black   
120                         Farberware® 120 Stainless Steel 1 qt. Saucepan in Aqua   
121       Farberware® Glide™ 10-Inch Nonstick Copper Ceramic Deep Skillet in Black   
124                              Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
125      Berndes® Vario® Click Induction Plus Nonstick 13-Inch Frying Pan in Black   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                      The Demi Tea Kettle from Le Creuset features a locking handle for added control and a heat-resistant phenolic knob handle for safe use. This enamel-on-steel tea kettle also features a harmonic whistle and can be used on all heat sources. Beautiful and eye-catching design would look great on any cooktop, kitchen counter, or coffee table Durable enamel-on-steel design with flat base for rapid heating Compatible with all heat sources, including induction Fixed harmonic whistle audibly alerts you when water has reached a roiling boil Spout easily flips open for easy pouring Locking handle with heat-resistant grip for added control and safe handling Wide, snug-fitting lid with phenolic knob handle for easy interior cleaning Enameled steel construction Hand wash 1.25 qt. capacity Measures 9.5\" L x 6\" W x 4.38\" H Weighs 2.4 lb. 5-year limited manufacturer's warranty Imported   
2    The enameled cast iron Le Creuset Signature Iron Handle Skillet offers even heat distribution and superb heat retention on all heat sources, including induction. Allo

In [33]:
matches['matches'][50:100]

67                          [Gas]
68                     [Electric]
69                    [Induction]
71                    [Induction]
72                    [Induction]
73                     [Electric]
74                    [Induction]
75                     [Electric]
78                    [Induction]
79                    [Induction]
80                    [Induction]
81                    [Induction]
82                    [Induction]
83                    [Induction]
84                    [Induction]
90                    [Induction]
91                    [Induction]
94                    [Induction]
98                    [Induction]
102                   [Induction]
103                   [Induction]
104                   [Induction]
107                   [Induction]
108    [Induction, Gas, Electric]
109                   [Induction]
115                   [Induction]
116                   [Induction]
117                   [Induction]
118                   [Induction]
120           

In [35]:
matches[curation_col] = matches['matches'].astype(str)
matches

product_id external_id  \
0    1211525005    69581629   
2    1174632103    69700513   
3     817553150    64521749   
4     817402026    68048099   
5     817187078    62290678   
..          ...         ...   
118  1184066770    18752697   
120  1192334030    69597868   
121  1184011997    66717515   
124   817164556    43267903   
125   988369545    64329406   

                                                                      product_name  \
0                       Le Creuset® Demi 1.25 qt. Whistling Tea Kettle in Licorice   
2                Le Creuset® Signature 10.25-Inch Iron Handle Skillet in Artichaut   
3        Ayesha Curry™ Cast Iron Enamel 12-Inch Skillet with Helper Handle in Teal   
4    Rachael Ray™ Create Delicious Nonstick 12.5-Inch Aluminum Deep Skillet in Red   
5                                 Rosle Stainless Steel Nonstick 11-Inch Crepe Pan   
..                                                                             ...   
118  Circulon® Symmetry™ Nonstick Hard Anodized 14-Inch Open Stir Fry Pan in Black   
120                         Farberware® 120 Stainless Steel 1 qt. Saucepan in Aqua   
121       Farberware® Glide™ 10-Inch Nonstick Copper Ceramic Deep Skillet in Black   
124                              Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
125      Berndes® Vario® Click Induction Plus Nonstick 13-Inch Frying Pan in Black   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                      The Demi Tea Kettle from Le Creuset features a locking handle for added control and a heat-resistant phenolic knob handle for safe use. This enamel-on-steel tea kettle also features a harmonic whistle and can be used on all heat sources. Beautiful and eye-catching design would look great on any cooktop, kitchen counter, or coffee table Durable enamel-on-steel design with flat base for rapid heating Compatible with all heat sources, including induction Fixed harmonic whistle audibly alerts you when water has reached a roiling boil Spout easily flips open for easy pouring Locking handle with heat-resistant grip for added control and safe handling Wide, snug-fitting lid with phenolic knob handle for easy interior cleaning Enameled steel construction Hand wash 1.25 qt. capacity Measures 9.5\" L x 6\" W x 4.38\" H Weighs 2.4 lb. 5-year limited manufacturer's warranty Imported   
2    The enameled cast iron Le Creuset Signature Iron Handle Skillet offers even heat distribution and superb heat retention on all heat sources, including induction. Allo

In [36]:
matches

product_id external_id  \
0    1211525005    69581629   
2    1174632103    69700513   
3     817553150    64521749   
4     817402026    68048099   
5     817187078    62290678   
..          ...         ...   
118  1184066770    18752697   
120  1192334030    69597868   
121  1184011997    66717515   
124   817164556    43267903   
125   988369545    64329406   

                                                                      product_name  \
0                       Le Creuset® Demi 1.25 qt. Whistling Tea Kettle in Licorice   
2                Le Creuset® Signature 10.25-Inch Iron Handle Skillet in Artichaut   
3        Ayesha Curry™ Cast Iron Enamel 12-Inch Skillet with Helper Handle in Teal   
4    Rachael Ray™ Create Delicious Nonstick 12.5-Inch Aluminum Deep Skillet in Red   
5                                 Rosle Stainless Steel Nonstick 11-Inch Crepe Pan   
..                                                                             ...   
118  Circulon® Symmetry™ Nonstick Hard Anodized 14-Inch Open Stir Fry Pan in Black   
120                         Farberware® 120 Stainless Steel 1 qt. Saucepan in Aqua   
121       Farberware® Glide™ 10-Inch Nonstick Copper Ceramic Deep Skillet in Black   
124                              Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
125      Berndes® Vario® Click Induction Plus Nonstick 13-Inch Frying Pan in Black   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                      The Demi Tea Kettle from Le Creuset features a locking handle for added control and a heat-resistant phenolic knob handle for safe use. This enamel-on-steel tea kettle also features a harmonic whistle and can be used on all heat sources. Beautiful and eye-catching design would look great on any cooktop, kitchen counter, or coffee table Durable enamel-on-steel design with flat base for rapid heating Compatible with all heat sources, including induction Fixed harmonic whistle audibly alerts you when water has reached a roiling boil Spout easily flips open for easy pouring Locking handle with heat-resistant grip for added control and safe handling Wide, snug-fitting lid with phenolic knob handle for easy interior cleaning Enameled steel construction Hand wash 1.25 qt. capacity Measures 9.5\" L x 6\" W x 4.38\" H Weighs 2.4 lb. 5-year limited manufacturer's warranty Imported   
2    The enameled cast iron Le Creuset Signature Iron Handle Skillet offers even heat distribution and superb heat retention on all heat sources, including induction. Allo

In [29]:
# error

In [18]:
# b_set = set(map(tuple,test_list))  #need to convert the inner lists to tuples so they are hashable
# b = map(list,b_set) #Now convert tuples back into lists (maybe unnecessary?)

In [37]:
# b_set

In [38]:
# def rem_dupes(dup_list): 
#     yooneeks = [] 
#     for elem in dup_list: 
#         if elem not in yooneeks: 
#             yooneeks.append(elem) 
#     return yooneeks

In [39]:
# rem_dupes(test_list[0:5])
# test_list[0:50]

In [40]:
# [list(set(sublist)) for sublist in test_list]

In [41]:
# test_list=test_list.reset_index()
# del test_list['index']
# test_list

In [42]:
# test_list[69]

In [43]:
# sorted(set(test_list))

In [44]:
# new_df=pd.DataFrame()
# for i in range(1):
#     new_df.append(sorted(set(test_list[i])))
# new_df

In [45]:
# test_list=lambda x: [list(set(test_list[i])) for i in test_list]
# test_list

In [46]:
# matches['matches'].astype(str).apply(lambda x: list(set(test_list[i])) for i in range(len(test_list)))

In [47]:
# error

In [48]:
# matches[curation_col] = matches['Max'].astype(str)+' in'
# # matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [49]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [50]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [51]:
looks_good('Bed Bath & Beyond', attribute, df, matches)